In [31]:
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split
from flaml import AutoML
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [32]:
def concat_csv_with_prefix(folder_path: str, prefix: str) -> pd.DataFrame:
    """
    Reads all CSV files in *folder_path* that start with *prefix*,
    concatenates them into a single DataFrame, and returns it.

    Parameters
    ----------
    folder_path : str
        Path to the directory containing the CSV files.
    prefix : str
        Prefix that the target CSV filenames must start with.

    Returns
    -------
    pd.DataFrame
        Concatenated DataFrame containing all rows from the matched files.
    """
    # Build glob pattern: e.g. "data/reviews_*.csv"
    pattern = os.path.join(folder_path, f"{prefix}*.csv")
    csv_files = sorted(glob.glob(pattern))

    if not csv_files:
        raise FileNotFoundError(f"No CSV files found with prefix '{prefix}' in {folder_path}")

    # Read each file into a DataFrame
    df_list = [pd.read_csv(f) for f in csv_files]

    # Concatenate all DataFrames, resetting the index
    return pd.concat(df_list, ignore_index=True)


In [33]:
df = concat_csv_with_prefix("data", "review")
df.head()

,product_name,product_category,review_size,type,content,review,stars
0,Samsung W-3000,WASHING_MACHINE,Long,Positive,['Functionality'],I recently upgraded to the Samsung W-3000 and ...,5
1,Panasonic W-7000,WASHING_MACHINE,Small,Negative,['Delivery' 'Delivery' 'Functionality'],I was disappointed with the Panasonic W-7000 b...,1
2,Electrolux L3-400,REFRIGERATOR,Long,Positive,['Price' 'Product appearance'],I was pleasantly surprised by the Electrolux L...,5
3,Panasonic Q2-300,REFRIGERATOR,Small,Positive,['Functionality'],The Panasonic Q2-300 keeps my groceries fresh ...,5
4,Pottery Barn S-2100,SOFA,Long,Negative,['Price' 'Functionality'],I was excited to add the Pottery Barn S-2100 t...,1


In [34]:
X = df[["review"]]
y = df["type"].apply(lambda x: 1 if x == "Positive" else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=20000, ngram_range=(1, 4))
X_train_vec = vectorizer.fit_transform(X_train["review"])
X_test_vec = vectorizer.transform(X_test["review"])

In [36]:
automl = AutoML()

settings = {
    "time_budget": 60,  # total running time in seconds
	"metric": 'accuracy',  # primary metric for optimization
	"task": 'classification',  # task type
	"log_file_name": "flaml.log",  # file to save log
}

automl.fit(X_train=X_train_vec, y_train=y_train, **settings)

[flaml.automl.logger: 02-12 16:50:32] {2375} INFO - task = classification
[flaml.automl.logger: 02-12 16:50:32] {2386} INFO - Evaluation method: holdout
[flaml.automl.logger: 02-12 16:50:32] {2489} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 02-12 16:50:32] {2606} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost', 'lrl1']
[flaml.automl.logger: 02-12 16:50:32] {2911} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 02-12 16:50:32] {3046} INFO - Estimated sufficient time budget=4915s. Estimated necessary time budget=121s.
[flaml.automl.logger: 02-12 16:50:32] {3097} INFO -  at 0.5s,	estimator lgbm's best error=1.7778e-01,	best estimator lgbm's best error=1.7778e-01
[flaml.automl.logger: 02-12 16:50:32] {2911} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 02-12 16:50:33] {3097} INFO -  at 0.6s,	estimator lgbm's best error=1.1667e-01,	best estimator lgbm's best error=1.16

In [37]:
print("Best model:", automl.model)
print("Best config:", automl.best_config)

Best model: SGDEstimator(_estimator_type='classifier',
             alpha=np.float64(4.0124290389139625e-06), average=False,
             epsilon=0.1, eta0=np.float64(0.028780808339510352),
             learning_rate='invscaling', loss='modified_huber', n_jobs=-1,
             penalty='l2', power_t=np.float64(0.4842406608374865),
             task=<flaml.automl.task.generic_task.GenericTask object at 0x7a63ae3c87a0>,
             tol=0.0001)
Best config: {'penalty': 'l2', 'alpha': np.float64(4.0124290389139625e-06), 'l1_ratio': np.float64(0.05350363127436544), 'epsilon': 0.1, 'learning_rate': 'invscaling', 'eta0': np.float64(0.028780808339510352), 'power_t': np.float64(0.4842406608374865), 'average': False, 'loss': 'modified_huber'}


In [38]:
pipeline = Pipeline([
    ("tfidf", vectorizer),
    ("clf", automl.model)
])


In [39]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("flaml_experiment")

with mlflow.start_run() as run:

    # Log parâmetros
    mlflow.log_params(automl.best_config)

    # Métrica
    preds = automl.predict(X_test_vec)
    acc = accuracy_score(y_test, preds)
    mlflow.log_metric("test_accuracy", acc)

    # Signature (muito importante)
    signature = infer_signature(X_train, pipeline.predict(X_train))

    # Log modelo
    mlflow.sklearn.log_model(
        pipeline,
        artifact_path="model",
        signature=signature
    )

    # Registrar no Model Registry
    model_uri = f"runs:/{run.info.run_id}/model"

    mlflow.register_model(
        model_uri=model_uri,
        name="flaml_model"
    )


Registered model 'flaml_model' already exists. Creating a new version of this model...
2026/02/12 16:51:39 WARNING mlflow.tracking._model_registry.fluent: Run with id 15ab23edf18c4523acfba246af9f52e1 has no artifacts at artifact path 'model', registering model based on models:/m-7c03057d21704b90ba21e30daabcb647 instead
2026/02/12 16:51:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: flaml_model, version 6
Created version '6' of model 'flaml_model'.


🏃 View run blushing-ox-696 at: http://127.0.0.1:5000/#/experiments/2/runs/15ab23edf18c4523acfba246af9f52e1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [50]:
pipeline.predict([X_test['review'].to_list()[0]])

array([0])

In [51]:
print(X_test['review'].to_list()[0])

Honestly, I'm pretty disappointed with the whole experience. The delivery was a complete nightmare - they were several hours late and didn't even call to let me know! And when they finally arrived, they practically threw the washing machine in, scratching the finish. It's a beautiful-looking appliance, I'll give it that, but for the price I paid, I expected so much more than this level of service and a pristine product. I'm seriously considering returning it.


In [56]:
pipeline.predict(["I love this product"])

array([1])